In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [15]:
DATA_DIR = "../input/clothing-dataset-full/images_original/"
DATA_RAW = "../input/clothing-dataset-full/images.csv"
MODEL_PATH = "../model.pth"
WIDTH, HEIGHT = 256, 256
BATCH_SIZE = 192
LEARNING_RATE = 1e-4
EPOCHS = 10
VALIDATION_SPLIT = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
data = pd.read_csv(DATA_RAW)
data = data.sample(len(data))
data.head(10)

,image,sender_id,label,kids
629,484ac9ae-b44b-4219-a58f-3911481c2e9d,171,Shoes,True
1196,4125a2d5-d084-4bde-94ce-819f7f39185f,313,Pants,False
2897,7dd5a8f9-4b36-41fc-84b2-3a3c96a1e7aa,267,Outwear,False
204,a021443c-ab32-41ff-8df5-cf24f6c5402c,359,T-Shirt,False
4231,a896e9f4-322d-4fd6-84eb-42acc7e3c344,181,Pants,False
3252,7cc4f821-7bff-467c-b8c7-6d32b5c9cc6c,260,Undershirt,False
1835,9d981826-b53b-449b-bf38-ead10ca80682,48,T-Shirt,False
5096,4efd804b-6c0e-48c5-9467-be5fcd5e3cf7,46,T-Shirt,False
620,1d2a178d-8e5a-4aa7-9801-b3eb3f0967b8,116,T-Shirt,False
263,c5924c48-00cc-43bd-b8a3-300a1ae7551f,102,Pants,False


In [17]:
data = pd.read_csv(DATA_RAW)
data = data.sample(len(data))
data.head(10)

,image,sender_id,label,kids
1570,459ec559-7f45-4906-95d7-d449d5eadfdf,370,Outwear,False
3140,1603b682-ed73-4f3e-88df-069d0efe2a55,48,T-Shirt,False
3500,c738675d-e3d7-415c-86c8-9dde4c209940,291,Shirt,False
2391,c3ebfb65-451d-4192-92f6-29d06875898f,262,Shirt,False
3854,5644e063-ad9d-4e12-896b-818a73bc3a15,54,Undershirt,False
3820,ad22e3b0-26a2-49cd-b46f-ea9a63c5df60,260,T-Shirt,False
2495,638f066a-218c-4850-9771-4883a908d6a3,181,T-Shirt,False
270,b19fb498-3efd-4e74-97e7-86e6843deb31,216,Longsleeve,True
384,5195e000-d919-4ebb-81b8-1e646224d231,167,Shoes,False
3818,84c1de9c-a87d-4c0f-bb3b-137e6b8309a6,267,Body,True


In [18]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((WIDTH, HEIGHT)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [20]:
from sklearn.model_selection import train_test_split
# Read and shuffle data
data = pd.read_csv(DATA_RAW)
data = data.sample(frac=1).reset_index(drop=True)
data["image_path"] = DATA_DIR + data["image"] + ".jpg"
# Map labels to integers
label_mapping = {label: idx for idx, label in enumerate(data["label"].unique())}
data["label"] = data["label"].map(label_mapping)

# Split data into training and validation sets using scikit-learn
train_df, val_df = train_test_split(data, test_size=VALIDATION_SPLIT, stratify=data["label"], random_state=42)

# Prepare dataset
class ClothingDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)  # Ensure indices are aligned
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = row["image_path"]
        label = row["label"]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Create dataset instances
train_dataset = ClothingDataset(train_df, transform=transform)
val_dataset = ClothingDataset(val_df, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [21]:
# Define the model
class ClothingClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ClothingClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32 * (WIDTH // 8) * (HEIGHT // 8), 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate model
num_classes = len(label_mapping)
model = ClothingClassifier(num_classes).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")

# Validation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Save the model
torch.save(model.state_dict(), MODEL_PATH)

Epoch [1/5], Loss: 3.8932
Epoch [2/5], Loss: 2.6107
Epoch [3/5], Loss: 2.5986
Epoch [4/5], Loss: 2.5601
Epoch [5/5], Loss: 2.4423
Validation Accuracy: 29.88%
